## Chapter 15: Scheduling Tasks

#### This chapter covers the creation and management of scheduled tasks in Snowflake. It introduces the creation of transient databases, task scheduling using both time-based and CRON expressions, and the organization of tasks into hierarchical structures. The chapter also explores tasks involving stored procedures and concludes with insights into monitoring task history.

### Creating Tasks

#### Creating a Transient Database and Table


In [ ]:
CREATE OR REPLACE TRANSIENT DATABASE TASK_DB;

CREATE OR REPLACE TABLE CUSTOMERS (
    CUSTOMER_ID INT AUTOINCREMENT START = 1 INCREMENT = 1,
    FIRST_NAME VARCHAR(40) DEFAULT 'JENNIFER',
    CREATE_DATE DATE);


#### Creating a Task


In [ ]:
CREATE OR REPLACE TASK CUSTOMER_INSERT
    WAREHOUSE = COMPUTE_WH
    SCHEDULE = '1 MINUTE'
    AS 
    INSERT INTO CUSTOMERS(CREATE_DATE) VALUES(CURRENT_TIMESTAMP);


#### Task Management


In [ ]:
SHOW TASKS;

ALTER TASK CUSTOMER_INSERT RESUME;
ALTER TASK CUSTOMER_INSERT SUSPEND;


### Using CRON


#### Using CRON Expressions in Task Scheduling


In [ ]:
CREATE OR REPLACE TASK CUSTOMER_INSERT
    WAREHOUSE = COMPUTE_WH
    SCHEDULE = 'USING CRON 0 7,10 * * 5L UTC'
    AS 
    INSERT INTO CUSTOMERS(CREATE_DATE) VALUES(CURRENT_TIMESTAMP);


#### CRON Examples


In [ ]:
SCHEDULE = 'USING CRON * * * * * UTC';  -- Every minute

SCHEDULE = 'USING CRON 0 6 * * * UTC';  -- Every day at 6am UTC timezone

SCHEDULE = 'USING CRON 0 9-17 * * SUN America/Los_Angeles';  -- Every hour starting at 9 AM and ending at 5 PM on Sundays


### Creating a Tree of Tasks


#### Creating a Tree of Tasks


In [ ]:
USE TASK_DB;

SHOW TASKS;

CREATE OR REPLACE TABLE CUSTOMERS2 (
    CUSTOMER_ID INT,
    FIRST_NAME VARCHAR(40),
    CREATE_DATE DATE);

CREATE OR REPLACE TASK CUSTOMER_INSERT2
    WAREHOUSE = COMPUTE_WH
    AFTER CUSTOMER_INSERT
    AS 
    INSERT INTO CUSTOMERS2 SELECT * FROM CUSTOMERS;

CREATE OR REPLACE TABLE CUSTOMERS3 (
    CUSTOMER_ID INT,
    FIRST_NAME VARCHAR(40),
    CREATE_DATE DATE,
    INSERT_DATE DATE DEFAULT DATE(CURRENT_TIMESTAMP));

CREATE OR REPLACE TASK CUSTOMER_INSERT3
    WAREHOUSE = COMPUTE_WH
    AFTER CUSTOMER_INSERT2
    AS 
    INSERT INTO CUSTOMERS3 (CUSTOMER_ID,FIRST_NAME,CREATE_DATE) SELECT * FROM CUSTOMERS2;

SHOW TASKS;

ALTER TASK CUSTOMER_INSERT SET SCHEDULE = '1 MINUTE';

ALTER TASK CUSTOMER_INSERT RESUME;
ALTER TASK CUSTOMER_INSERT2 RESUME;
ALTER TASK CUSTOMER_INSERT3 RESUME;


### Task with Stored Procedure


#### Creating a Task with a Stored Procedure


In [ ]:
USE TASK_DB;

SELECT * FROM CUSTOMERS;

CREATE OR REPLACE PROCEDURE CUSTOMERS_INSERT_PROCEDURE (CREATE_DATE varchar)
    RETURNS STRING NOT NULL
    LANGUAGE JAVASCRIPT
    AS
        $$
        var sql_command = 'INSERT INTO CUSTOMERS(CREATE_DATE) VALUES(:1);'
        snowflake.execute(
            {
            sqlText: sql_command,
            binds: [CREATE_DATE]
            });
        return "Successfully executed.";
        $$;


### Task History


#### Viewing Task History


In [ ]:
SHOW TASKS;

USE DEMO_DB;

-- Use the table function "TASK_HISTORY()"
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY())
ORDER BY SCHEDULED_TIME DESC;

-- See results for a specific Task in a given time
SELECT *
FROM TABLE(INFORMATION_SCHEMA.TASK_HISTORY(
    SCHEDULED_TIME_RANGE_START => DATEADD('HOUR', -4, CURRENT_TIMESTAMP()),
    RESULT_LIMIT => 5,
    TASK_NAME => 'CUSTOMER_INSERT2'));
